In [3]:
import torch as t
import numpy as np
import pandas as pd

In [8]:
class Cerberus (t.nn.Module):
    def __init__(self, config):
        super().__init__()

        self.encoder = t.nn.Sequential(
            t.nn.Conv1d(config["input"], config["hidden1"], config["kern_size1"], config["padding1"], config["dilation1"]),
            t.nn.ReLU(),
            t.nn.Conv1d(config["hidden1"], config["hidden1"], config["kern_size1"], config["padding2"], config["dilation2"]),
        )

        #x, y, z
        self.position_head = t.nn.sequential(
            t.nn.Linear(config["hidden1"], config["pos_head_sz"]),
            t.nn.ReLU(),
            t.nn.Linear(config["pos_head_sz"], 3)
        )

        #speed, gas, brake, side-left, side-right
        self.control_head = t.nn.sequential(
            t.nn.Linear(config["hidden1"], config["cont_head_sz"]),
            t.nn.ReLU(),
            t.nn.Linear(config["cont_head_sz"], 5)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        encoded = encoded[:, :, -1]

        position = self.pos_head(encoded)
        control_in = t.cat([encoded, position], dim=1)
        control = self.control_head(control_in)

        
        return position, control

In [9]:
class PositionLoss(t.nn.Module):
    def __init__(self, weight=1.0):
        super(PositionLoss, self).__init__()
        self.mse_loss = t.nn.MSELoss()

    def forward(self, predicted, target):
        loss = self.mse_loss(predicted, target)
        return loss

#we can add penalties to encourage smoother and more realistic driving here
class ControlLoss(t.nn.Module):
    def __init__(self, weight=1.0):
        super(ControlLoss, self).__init__()
        self.mse_loss(predicted, target)
        return loss